In [31]:
#tensorflow的一些操作
import tensorflow as tf
import numpy as np
t = tf.constant([[1,2,3],[4,5,6]])
t_T = tf.transpose(t)
print(t)
print(t.shape)   
print(t.dtype)   
print(t[:,1:]) 
print(t+10)              #张量
print(tf.square(t))      #平方
print(tf.transpose(t))   #转置 
print(tf.matmul(t,t_T))  #矩阵乘法
print(tf.exp(2.0))
print(tf.sqrt(2.0)) #不能为整型
print(tf.reshape(t,shape=[3,2]))
print(tf.reduce_sum(t))
print(tf.math.log(2.0))
print(t.numpy())

#类型转换
t2 = tf.constant(40.,dtype=tf.float64)
tf.constant(2.0)+tf.cast(t2,dtype=tf.float32)

#变量
l = tf.constant([[1,2,3],[4,5,6]])
v = tf.Variable(l)
print(v[1,1].assign(42))
v.scatter_nd_update(indices=[[0,0],[1,2]],updates=[100,200])

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
(2, 3)
<dtype: 'int32'>
tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[11 12 13]
 [14 15 16]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[1 4]
 [2 5]
 [3 6]], shape=(3, 2), dtype=int32)
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)
tf.Tensor(7.389056, shape=(), dtype=float32)
tf.Tensor(1.4142135, shape=(), dtype=float32)
tf.Tensor(
[[1 2]
 [3 4]
 [5 6]], shape=(3, 2), dtype=int32)
tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(0.6931472, shape=(), dtype=float32)
[[1 2 3]
 [4 5 6]]
<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 1,  2,  3],
       [ 4, 42,  6]])>


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[100,   2,   3],
       [  4,  42, 200]])>

In [ ]:
#定制模型和训练方法
import tensorflow as tf
from tensorflow import keras
def huber_fn(y_true,y_pred):
    error = y_true-y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-0.5
    return tf.where(is_small_error,squared_loss,linear_loss)

a = tf.constant([1,2,3,0])
b = tf.constant([0,3,2,1])
tf.where(a>1,a,b)   #where返回索引  根据a>1返回a或者b的值


#自定义损失函数
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold=1.0,**kwargs):
        super.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true-y_pred
        is_small_error = tf.abs(error)<self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error)-self.threshold**/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,'threshold':self.threshold}

# model.compile(loss=HuberLoss(2.),optimizer='nadam')
# model = keras.models.load_model('my_model_with_a_custom_loss_class.h5',custom_objects={'HuberLoss':HuberLoss})



#自定义激活函数,初始化，正则化，约束
def my_softplus(z):
    return tf.math.log(tf.exp(z)+1.0)   #return value is just tf.nn.softplus(z)
def my_glorot_initializer(shape,dtype=tf.float32):
    stddev = tf.sqrt(2./(shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01*weights))
def my_positive_weights(weights):
    return tf.where(weights<0,tf.zeros_like(weights),weights)

layer = keras.layers.Dense(30,activation=my_softplus,
                          kernel_initializer=my_glorot_initializer,
                          kernel_regularizer=my_l1_regularizer,
                          kernel_constraint=my_positive_weights)

In [6]:
#自定义模型
from tensorflow import keras
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_layers,n_neurons,**kwargs):
        super().init(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons,activation='elu',kernel_initializer='he_normal') for _ in range(n_layers)]
    def call(self,inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return inputs+z
class ResidualRegressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30,activation='elu',kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        self.out = keras.layers.Dense(output_dim)
    def call(self,inputs):
        z = self.hidden1(inputs)
        for _ in range(1+3):
            z = self.block1(z)
        z = self.block2(z)
        return self.out(z)
# ResidualRegressor.compile()
# ResidualRegressor.fit()
# ResidualRegressor.evaluate()
# ResidualRegressor.predict()

In [ ]:
#基于模型内部的损失和指标(重建损失)
class ReconstructingRegressor(keras.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30,activation='selu'kernel_initializer='lecun_normal') for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
    def build(self,batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)
    def call(self,inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        reconstruction = self.reconstruct(z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction-inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(z)
    

In [11]:
#tensorflow函数和图
import tensorflow as tf
import time
def f(x):
    return x**3
print(f(3))

tf_f = tf.function(f)
print(tf_f(3))

print(tf_f.python_function(3))

27
tf.Tensor(27, shape=(), dtype=int32)
27
